# Preprocesamiento

In [ ]:
import pandas as pd

import sys
sys.path.append("..")

from src.support_prep import *
from src.support_models import *    

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_pickle("../datos/clean.pkl")

In [6]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [11]:
cat_cols = df.select_dtypes("O").columns.drop("Attrition")

In [12]:
detectar_orden_cat(df = df, lista_cat=cat_cols,var_respuesta="Attrition")

Estamos evaluando la variable BUSINESSTRAVEL


Attrition,No,Yes
BusinessTravel,,
Non-Travel,414,36
Travel_Frequently,624,207
Travel_Rarely,2661,468


Para la categoría BUSINESSTRAVEL SÍ hay diferencias significativas, p = 0.0000


Attrition,No,Yes
BusinessTravel,,
Non-Travel,377.0,73.0
Travel_Frequently,697.0,134.0
Travel_Rarely,2625.0,504.0


--------------------------------------------------------------------------------
Estamos evaluando la variable DEPARTMENT


...

--------------------------------------------------------------------------------
Estamos evaluando la variable WORKLIFEBALANCE


Attrition,No,Yes
WorkLifeBalance,,
1.0,164,75
2.0,848,171
3.0,2280,380
4.0,373,81
unknown,34,4


Para la categoría WORKLIFEBALANCE SÍ hay diferencias significativas, p = 0.0000


Attrition,No,Yes
WorkLifeBalance,,
1.0,200.0,39.0
2.0,855.0,164.0
3.0,2231.0,429.0
4.0,381.0,73.0
unknown,32.0,6.0


--------------------------------------------------------------------------------


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from imblearn.under_sampling import TomekLinks


# categorical_columns = df_general.drop(columns="Attrition").select_dtypes(include=["object", "category"]).columns
# df_encoded = df_general.copy()

# label_encoders = {}
# for col in categorical_columns:
#     le = LabelEncoder()
#     df_encoded[col] = le.fit_transform(df_encoded[col])
#     label_encoders[col] = le

# X = df_encoded.drop(columns="Attrition")
# y = df_encoded["Attrition"]

# from imblearn.combine import SMOTETomek
# smote_tomek = SMOTETomek(random_state=42)
# X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

# for col in categorical_columns:
#     X_resampled[col] = label_encoders[col].inverse_transform(X_resampled[col])